In [2]:
!pwd
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

/home/tomi/code/Harlequinht/which_horse/notebook


In [3]:
from pipeline_cleaning import clean_data
from pipeline_cleaning import transforming_data
# from custom_loss_not_nn import custom_loss_function_not_nn
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
import numpy as np

In [4]:
def custom_loss_function_not_nn(y_true, y_pred):
    return -0.1 * K.mean(K.sum(gain_loss_vector * y_pred, axis=1))

In [5]:
from sklearn.metrics import make_scorer
custom_loss = make_scorer(custom_loss_function_not_nn, greater_is_better=False)

In [6]:
df = pd.read_csv('../raw_data/combined_flat2_csv.csv')
df['randNumCol'] = np.random.choice([1, 9, 20], df.shape[0])


/tmp/ipykernel_188931/127687269.py:1: DtypeWarning: Columns (22,53,62,71,80,89,98,107,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/combined_flat2_csv.csv')


In [7]:
df_clean= clean_data(df)
df_transform = transforming_data(df_clean)
train = df_transform[0]
test = df_transform[2]

/home/tomi/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['failed_to_finish_reason'] = df['failed_to_finish_reason'].apply(lambda x: 0 if pd.isna(x) else 1)
/home/tomi/code/Harlequinht/which_horse/notebook/pipeline_cleaning.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['margin'] = df.apply(lambda row: row['distance'] if pd.isna(row['margin']) and (row['win_or_lose'] == 1 or row['failed_to_finish_reason'] == 1) else row['margin'], axi

In [8]:
X_train = train.drop(columns = ['win_or_lose', 'randNumCol'], axis =1)
X_test = test.drop(columns = ['win_or_lose', 'randNumCol'], axis =1 )
y_train = train[['win_or_lose']]
y_test = test[['win_or_lose']]

In [13]:
models = [
        # LinearRegression(),
        #   Ridge(),
        #   Lasso(),
        #   ElasticNet(),
          # SGDRegressor(loss = custom_loss_function_not_nn),
        #   KNeighborsRegressor(),
        #   SVR(kernel = "linear"),
        #   SVR(kernel = "poly", degree = 2),
        #   SVR(kernel = "poly", degree = 3),
        #   SVR(kernel = "rbf"),
        #   DecisionTreeRegressor(),
        #   RandomForestRegressor(),
          AdaBoostRegressor(loss = custom_loss_function_not_nn),
          GradientBoostingRegressor(loss = custom_loss_function_not_nn)]

models_names = ["ada_boost_regressor",
                "gradient_boosting_regressor"]


In [15]:
different_test_scores = {}
    
for model_name, model in zip(models_names, models):

    model.fit(X_train, y_train)
    different_test_scores[model_name] = model.score(X_test, y_test)
    
    
comparing_regression_models = pd.DataFrame(list(zip(models_names, different_test_scores)),
                                                columns =['model_name', 'test_score'])

print(round(comparing_regression_models.sort_values(by = "test_score", ascending = False),2))

InvalidParameterError: The 'loss' parameter of AdaBoostRegressor must be a str among {'linear', 'exponential', 'square'}. Got <function custom_loss_function_not_nn at 0x7f5f56b9c280> instead.